In [1]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import db_dtypes

In [2]:
client = bigquery.Client(project='first-project-321219')

query = """
WITH session_windows AS (
    -- Step 1: Define the start and end of every session
    SELECT 
        session_id,
        user_id,
        MIN(created_at) AS session_start,
        MAX(created_at) AS session_end
    FROM `bigquery-public-data.thelook_ecommerce.events`
    WHERE created_at >= '2022-01-01'
    GROUP BY 1, 2
),
order_totals AS (
    -- Step 2: Get total value for every order
    SELECT 
        user_id,
        order_id,
        created_at,
        SUM(sale_price) AS total_value
    FROM `bigquery-public-data.thelook_ecommerce.order_items`
    GROUP BY 1, 2, 3
)

SELECT 
    sw.session_id,
    sw.user_id,
    u.age,
    u.gender,
    u.country,
    u.traffic_source,
    sw.session_start,

    -- FEATURE: Total spend BEFORE this session started
    COALESCE((
        SELECT SUM(ot.total_value)
        FROM order_totals ot
        WHERE ot.user_id = sw.user_id 
          AND ot.created_at < sw.session_start
    ), 0) AS past_total_spend_before_session,

    -- FEATURE: Session count prior to this one
    COUNT(*) OVER (
        PARTITION BY sw.user_id 
        ORDER BY sw.session_start 
        ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
    ) AS number_of_prior_session_count,

    -- TARGET: Total spend WITHIN this session
    -- We look for orders by this user that happened between the session start and end
    COALESCE((
        SELECT SUM(ot.total_value)
        FROM order_totals ot
        WHERE ot.user_id = sw.user_id 
          AND ot.created_at BETWEEN sw.session_start AND sw.session_end
    ), 0) AS label_session_spend

FROM session_windows sw
JOIN `bigquery-public-data.thelook_ecommerce.users` u ON sw.user_id = u.id
ORDER BY sw.user_id, sw.session_start
"""

df = client.query(query).to_dataframe()
print(df.head())

                             session_id  user_id  age gender        country  \
0  53f4212d-da3c-413c-8009-2c362540df67        1   26      F  United States   
1  d815acfc-2672-4c85-9aad-a49a5cd1c7a8        2   17      M          Japan   
2  142f1d03-18e6-45c5-b351-b18bdd1b6c8e        3   16      M      Australia   
3  4306846f-3ded-43aa-ab67-ae1458315303        3   16      M      Australia   
4  7bfd934f-edbd-4568-981a-4af43be5bed0        4   69      F         Brasil   

  traffic_source             session_start  past_total_spend_before_session  \
0         Search 2024-09-08 16:47:28+00:00                              0.0   
1         Search 2024-12-13 22:06:40+00:00                              0.0   
2         Search 2024-11-21 13:19:14+00:00                              0.0   
3         Search 2024-11-21 14:26:51+00:00                              0.0   
4        Organic 2023-12-05 06:41:13+00:00                              0.0   

   number_of_prior_session_count  label_session_sp

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162898 entries, 0 to 162897
Data columns (total 10 columns):
 #   Column                           Non-Null Count   Dtype              
---  ------                           --------------   -----              
 0   session_id                       162898 non-null  object             
 1   user_id                          162898 non-null  Int64              
 2   age                              162898 non-null  Int64              
 3   gender                           162898 non-null  object             
 4   country                          162898 non-null  object             
 5   traffic_source                   162898 non-null  object             
 6   session_start                    162898 non-null  datetime64[us, UTC]
 7   past_total_spend_before_session  162898 non-null  float64            
 8   number_of_prior_session_count    162898 non-null  Int64              
 9   label_session_spend              162898 non-null  float64  

In [4]:
def load_data(dataframe, datefrom = None, dateto = None):
    
    if not isinstance(dataframe, pd.DataFrame):   #isinstance(): check specified object is in specificed type.
        print('Error: Input is not a DataFrame')
        return None
  

    if 'label_session_spend' not in dataframe.columns: 
        print(f'Column Error: label_total_spend is not in DataFrame')
        return dataframe
        
    try:
        dataframe['session_start'] = pd.to_datetime(dataframe['session_start'])
    except Exception:
        print(f'Date Conversion Error: {Exception}')

    return dataframe

    

In [5]:
df_raw = load_data(df)
df_raw.head()

,session_id,user_id,age,gender,country,traffic_source,session_start,past_total_spend_before_session,number_of_prior_session_count,label_session_spend
0,53f4212d-da3c-413c-8009-2c362540df67,1,26,F,United States,Search,2024-09-08 16:47:28+00:00,0.0,0,0.000000
1,d815acfc-2672-4c85-9aad-a49a5cd1c7a8,2,17,M,Japan,Search,2024-12-13 22:06:40+00:00,0.0,0,0.000000
2,142f1d03-18e6-45c5-b351-b18bdd1b6c8e,3,16,M,Australia,Search,2024-11-21 13:19:14+00:00,0.0,0,189.949997
3,4306846f-3ded-43aa-ab67-ae1458315303,3,16,M,Australia,Search,2024-11-21 14:26:51+00:00,0.0,1,449.939987
4,7bfd934f-edbd-4568-981a-4af43be5bed0,4,69,F,Brasil,Organic,2023-12-05 06:41:13+00:00,0.0,0,0.000000


In [6]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162898 entries, 0 to 162897
Data columns (total 10 columns):
 #   Column                           Non-Null Count   Dtype              
---  ------                           --------------   -----              
 0   session_id                       162898 non-null  object             
 1   user_id                          162898 non-null  Int64              
 2   age                              162898 non-null  Int64              
 3   gender                           162898 non-null  object             
 4   country                          162898 non-null  object             
 5   traffic_source                   162898 non-null  object             
 6   session_start                    162898 non-null  datetime64[us, UTC]
 7   past_total_spend_before_session  162898 non-null  float64            
 8   number_of_prior_session_count    162898 non-null  Int64              
 9   label_session_spend              162898 non-null  float64  

In [14]:
df_raw[df_raw['label_session_spend']==0]
# 78457 / 162898 = 48%, close to half of the customers spent zero. it can cause skewed distribution

,session_id,user_id,age,gender,country,traffic_source,session_start,past_total_spend_before_session,number_of_prior_session_count,label_session_spend,day_num,is_weekend
0,53f4212d-da3c-413c-8009-2c362540df67,1,26,F,United States,Search,2024-09-08 16:47:28+00:00,0.000000,0,0.0,6,1
1,d815acfc-2672-4c85-9aad-a49a5cd1c7a8,2,17,M,Japan,Search,2024-12-13 22:06:40+00:00,0.000000,0,0.0,4,0
4,7bfd934f-edbd-4568-981a-4af43be5bed0,4,69,F,Brasil,Organic,2023-12-05 06:41:13+00:00,0.000000,0,0.0,1,0
8,04f559b1-750e-4dc9-ae4a-da70ac1d6e4a,6,31,M,United States,Organic,2022-04-18 22:06:11+00:00,0.000000,0,0.0,0,0
9,da99225c-237b-4af8-af4c-41e515a2798c,9,37,M,China,Organic,2024-05-18 01:00:54+00:00,0.000000,0,0.0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...
162871,5ea09ef4-618c-40e0-83bd-17444b4f8b0d,99988,30,M,China,Search,2024-10-06 20:12:05+00:00,0.000000,0,0.0,6,1
162872,bdf71072-3dcd-41d7-b7fa-46fc49a93ae7,99988,30,M,China,Search,2025-07-17 21:19:58+00:00,39.950001,1,0.0,3,0
162873,ff880e63-13ac-4d02-bf17-fe8b02593230,99989,20,M,United States,Organic,2023-01-21 13:13:41+00:00,0.000000,0,0.0,5,1
162874,694ce09e-c480-482a-872c-4dfbb8466fa0,99993,46,M,France,Search,2025-02-22 23:00:14+00:00,0.000000,0,0.0,5,1


In [7]:
def prepare_data(df_input, target = 'label_session_spend'):

    df_input['day_num'] = df_input['session_start'].dt.weekday
    df_input['is_weekend']= (df_input['day_num'] >=5).astype(int)

    df_input['country'] = df_input['country'].replace(
                                            {'España': 'Spain', 
                                             'Deutschland': 'Germany'},regex = False)

    df_proc = df_input.copy()
    df_proc = df_proc.drop(['session_id', 'user_id', 'session_start'], axis = 1)
                            
    cols_obj = df_proc.select_dtypes(include=['O']).columns.tolist()

    df_getdum = pd.get_dummies(df_proc, columns = cols_obj, drop_first = True, dtype = int)

    #Reorder columns: put target at the end
    if target in df_getdum.columns:
        cols = [c for c in df_getdum.columns if c not in target]
        df_getdum = df_getdum[cols + [target]]

    return df_getdum
    
    

In [8]:
df_prepare = prepare_data(df_raw)
df_prepare

,age,past_total_spend_before_session,number_of_prior_session_count,day_num,is_weekend,gender_M,country_Austria,country_Belgium,country_Brasil,country_China,...,country_Poland,country_South Korea,country_Spain,country_United Kingdom,country_United States,traffic_source_Email,traffic_source_Facebook,traffic_source_Organic,traffic_source_Search,label_session_spend
0,26,0.000000,0,6,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0.000000
1,17,0.000000,0,4,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.000000
2,16,0.000000,0,3,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,189.949997
3,16,0.000000,1,3,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,449.939987
4,69,0.000000,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162893,54,108.979998,3,6,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,69.949997
162894,54,199.879995,4,6,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,46.000000
162895,54,199.879995,5,6,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,105.980000
162896,54,199.879995,6,6,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,81.990000


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

def linear_regression(df, target = 'label_session_spend', test_size = 0.2, random_state = 16):
    # age, number_of_prior_session_count are Int64, This is a special Pandas "Nullable Integer" type. While it looks like a number to us, statsmodels and numpy.isfinite often struggle with it because it's technically an object that allows for NaN values.
    # convert to Pandas-style integers
    df = df.astype(float)
    
    X = df.drop(['day_num', 'label_session_spend'], axis=1).select_dtypes(include=[np.number])
    X = X.astype(float)

    if X.isnull().values.any():
        X = X.fillna(0)
        print('Nan value found and fill with 0')
    y = df[target]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = random_state)

 
    scaler = StandardScaler()

    X_train_scaled = scaler.fit_transform(X_train)

    X_test_scaled = scaler.transform(X_test)

    linreg = LinearRegression()

    linreg.fit(X_train_scaled, y_train)

    y_pred = linreg.predict(X_test_scaled)

    #Evaluate model performance

    r2 = r2_score(y_test, y_pred)
    print(f'R_Squared: {r2:4f}')

    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error: {mse:4f}')

    rmse = mse ** 0.5
    print(f'Root Mean Squared Error: {rmse:4f}')

    #evaluate multicollinearity
    corr_matrix = X.corr()

    high_corr_feature = [(col1, col2, corr_matrix.loc[col1, col2])
                         for col1 in corr_matrix.columns
                         for col2 in corr_matrix.columns
                         if col1 != col2 and abs(corr_matrix.loc[col1, col2]) > 0.8]

    high_corr_df = pd.DataFrame(high_corr_feature, columns=['Feature1', 'Feature2', 'Correlation'])
    print('High Correlated Features:', high_corr_df)

    # statsmodels VIF needs a constant (intercept) to be accurate  

    X_vif = add_constant(X)
    vif_data = pd.DataFrame()
    vif_data['Feature'] = X_vif.columns
    vif_data['VIF'] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

    print('\nVariance infaltion Factor(VIF) for each feature:\n', vif_data)

    return linreg

In [10]:
df_linreg = linear_regression(df_prepare)
# High ocrrelated Features return Empty means none of the features have high correlation.

R_Squared: 0.071141
Mean Squared Error: 7460.002981
Root Mean Squared Error: 86.371309
High Correlated Features: Empty DataFrame
Columns: [Feature1, Feature2, Correlation]
Index: []

Variance infaltion Factor(VIF) for each feature:
                             Feature        VIF
0                             const  76.568010
1                               age   1.000334
2   past_total_spend_before_session   1.720404
3     number_of_prior_session_count   1.718693
4                        is_weekend   1.000357
5                          gender_M   1.002243
6                   country_Austria   1.000754
7                   country_Belgium   1.578834
8                    country_Brasil   6.350882
9                     country_China  10.627518
10                 country_Colombia   1.002906
11                   country_France   2.944925
12                  country_Germany   2.725722
13                    country_Japan   2.028487
14                   country_Poland   1.098140
15             

In [11]:
print(df_prepare.dtypes)

age                                  Int64
past_total_spend_before_session    float64
number_of_prior_session_count        Int64
day_num                              int32
is_weekend                           int64
gender_M                             int64
country_Austria                      int64
country_Belgium                      int64
country_Brasil                       int64
country_China                        int64
country_Colombia                     int64
country_France                       int64
country_Germany                      int64
country_Japan                        int64
country_Poland                       int64
country_South Korea                  int64
country_Spain                        int64
country_United Kingdom               int64
country_United States                int64
traffic_source_Email                 int64
traffic_source_Facebook              int64
traffic_source_Organic               int64
traffic_source_Search                int64
label_sessi

In [12]:
df_prepare.isnull().values.any()

np.False_